In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import os

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from src.utils.time_series_procs import split_sequences
from src.models.transformer_ts_tf import Transformer

In [2]:
def get_times_series_data(df, cut_date):
    columns = list(df.columns.drop(["R28D"]))
    columns.append("R28D")
    df = df.reindex(columns=columns)
    df["Data/Hora"] = pd.to_datetime(df["Data/Hora"])
    df_train = df[df["Data/Hora"] < cut_date]
    df_test = df[df["Data/Hora"] >= cut_date]
    df_train = df_train.drop(["Data/Hora"], axis=1)
    df_test = df_test.drop(["Data/Hora"], axis=1)
    x_train, y_train = split_sequences(df_train.values, TIME_STEPS)
    x_test, y_test = split_sequences(df_test.values, TIME_STEPS)
    return x_train, y_train, x_test, y_test

In [3]:
class StandardScaler3DShape:
    def __init__(self):
        self.scaler = StandardScaler()

    def fit_transform(self, X, y=None):
        X_new = self.scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        return X_new

    def fit(self, X, y=None):
        X_new = X.reshape(-1, X.shape[-1])
        self.scaler.fit(X_new)
        return self

    def transform(self, X, y=None):
        X_new = self.scaler.transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        return X_new

In [4]:
def get_baseline_model(
    num_features, time_steps, num_heads=2, num_layers_enc=1, num_feed_forward=32
):
    model = Transformer(
        num_hid=num_features,
        time_steps=time_steps,
        num_head=num_heads,
        num_layers_enc=num_layers_enc,
        num_feed_forward=num_feed_forward,
    )

    opt = tf.keras.optimizers.Adam(learning_rate=0.01)
    model.compile(
        optimizer=opt,
        loss="mse",
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
    )
    return model

In [5]:
def make_repeated_time_series_k_fold(
    x,
    y,
    train_period,
    test_period,
    repeats=10,
    show_individual_results=True,
    show_final_result=True,
):
    results = []
    max_samples = x.shape[0]

    for _ in range(repeats):
        i = 0
        scores = []
        for _ in range(0, max_samples // train_period):
            x_train = x[i : i + train_period]
            y_train = y[i : i + train_period]
            x_test = x[i + train_period : i + train_period + test_period]
            y_test = y[i + train_period : i + train_period + test_period]
            i += train_period
            x_train = scaler.fit_transform(
                x_train.reshape(-1, x_train.shape[-1])
            ).reshape(x_train.shape)
            x_test = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(
                x_test.shape
            )
            model = get_baseline_model(
                num_features=x_train.shape[-1], time_steps=x_train.shape[-2]
            )
            model.fit(x_train, y_train, epochs=100, batch_size=32, verbose=0)
            rmse = model.evaluate(x_test, y_test, verbose=0)
            scores.append(rmse[1])
        results.append(scores)
        if show_individual_results:
            print("RMSE: %.3f (%.3f)" % (np.mean(scores), np.std(scores)))
    if show_final_result:
        print("\nRMSE: %.3f (%.3f)" % (np.mean(results), np.std(results)))
    return results

In [6]:
def make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats=10, timesteps_list=[3], show_results=True
):
    results = {}

    for timesteps in timesteps_list:
        x, y = split_sequences(df.values, timesteps)
        scores = make_repeated_time_series_k_fold(
            x,
            y,
            train_period,
            test_period,
            repeats=10,
            show_individual_results=False,
            show_final_result=False,
        )
        results[timesteps] = scores
        if show_results:
            print(
                "TIMESTEPS: %d RMSE: %.3f (%.3f)"
                % (timesteps, np.mean(scores), np.std(scores))
            )
    return results

<h3>Disable Tensorflow Warnings</h3>

In [7]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [8]:
df_r3d_and_r7d = pd.read_csv(
    "../../../../data/interim/"
    + "r3d_and_r7d-no-resampling-with-fillna-"
    + "dados-historicos-partner_i-cement-CPIIF40.csv"
)
df_r3d_only = pd.read_csv(
    "../../../../data/interim/"
    + "r3d_only-no-resampling-with-fillna-"
    + "dados-historicos-partner_i-cement-CPIIF40.csv"
)
df_no_r3d_r7d = pd.read_csv(
    "../../../../data/interim/"
    + "no-r3d-r7d-no-resampling-with-fillna-"
    + "dados-historicos-partner_i-cement-CPIIF40.csv"
)

In [9]:
seed = 47
scaler = StandardScaler()

timesteps_list = [1, 3, 5, 7, 10, 15, 20]
repeats = 10

## Feature Selection

### In this notebook we remove variables that could potentially cause data leakage. Thus we are performing testings removing the following variables:

#### IP - Initial setting time
#### FP - Final setting time


#### BL - Blaine specific surface (This is both removed and use)

# Transformer Encoder

# Configs:

1. TIMESTEPS: [1, 3, 5, 7, 10, 15, 20]

2. Epochs: 100

3. Batch size: 32

4. num heads: 2

5. FF Layers enc: 1

      5.1 Units 32

6. Optimization Algorithm: Adam
    
    6.1. Learning Rate: 1e-2

# Times Series Repeated KFold Cross validation - different Timesteps values

Here we will make a Times Series KFold Cross validation with all 3 datasets. Here we are using the Blocking Time Series Split method. And with timeseries Split method. We also do it with different number of timesteps


## 1.1 Dataset: no-r3d-r7d-no-resampling-with-fillna

<h3>1.1.1 Time Series Repeated KFold Cross Validation with vary timesteps - df_no_r3d_r7d</h3>

<b>Dataset: df_no_r3d_r7d:</b> In this dataset the R3D and R7D variables are not considered.

<b>Feature Selection:</b> The IP and FP variables are discarded.

In [10]:
df = df_no_r3d_r7d.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora', "IP", "FP"], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [11]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [12]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.010 (0.244)
TIMESTEPS: 3 RMSE: 2.030 (0.210)
TIMESTEPS: 5 RMSE: 1.952 (0.371)
TIMESTEPS: 7 RMSE: 2.246 (0.538)
TIMESTEPS: 10 RMSE: 2.327 (0.424)
TIMESTEPS: 15 RMSE: 2.949 (0.653)
TIMESTEPS: 20 RMSE: 2.755 (0.513)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [13]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [14]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.073 (0.238)
TIMESTEPS: 3 RMSE: 2.448 (0.495)
TIMESTEPS: 5 RMSE: 2.406 (0.526)
TIMESTEPS: 7 RMSE: 2.229 (0.595)
TIMESTEPS: 10 RMSE: 2.619 (0.662)
TIMESTEPS: 15 RMSE: 2.071 (0.115)
TIMESTEPS: 20 RMSE: 2.742 (0.327)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [15]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [16]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.857 (0.342)
TIMESTEPS: 3 RMSE: 3.392 (0.586)
TIMESTEPS: 5 RMSE: 3.679 (0.516)
TIMESTEPS: 7 RMSE: 3.361 (0.405)
TIMESTEPS: 10 RMSE: 3.333 (0.327)
TIMESTEPS: 15 RMSE: 3.118 (0.392)
TIMESTEPS: 20 RMSE: 2.926 (0.365)


<h3>1.1.2 Time Series Repeated KFold Cross Validation with vary timesteps - df_no_r3d_r7d</h3>

<b>Dataset: df_no_r3d_r7d:</b> In this dataset the R3D and R7D variables are not considered.

<b>Feature Selection:</b> The IP, FP and BL variables are discarded.

In [17]:
df = df_no_r3d_r7d.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora', "IP", "FP", "BL"], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [18]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [19]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 1.987 (0.228)
TIMESTEPS: 3 RMSE: 2.102 (0.491)
TIMESTEPS: 5 RMSE: 2.043 (0.275)
TIMESTEPS: 7 RMSE: 2.274 (0.616)
TIMESTEPS: 10 RMSE: 2.268 (0.416)
TIMESTEPS: 15 RMSE: 3.036 (0.668)
TIMESTEPS: 20 RMSE: 2.745 (0.383)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [20]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [21]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.079 (0.363)
TIMESTEPS: 3 RMSE: 2.139 (0.392)
TIMESTEPS: 5 RMSE: 2.474 (0.389)
TIMESTEPS: 7 RMSE: 2.513 (0.438)
TIMESTEPS: 10 RMSE: 2.471 (0.411)
TIMESTEPS: 15 RMSE: 2.033 (0.119)
TIMESTEPS: 20 RMSE: 2.821 (0.321)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [22]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [23]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.728 (0.196)
TIMESTEPS: 3 RMSE: 2.994 (0.213)
TIMESTEPS: 5 RMSE: 3.227 (0.412)
TIMESTEPS: 7 RMSE: 3.369 (0.425)
TIMESTEPS: 10 RMSE: 3.298 (0.389)
TIMESTEPS: 15 RMSE: 3.167 (0.580)
TIMESTEPS: 20 RMSE: 2.885 (0.394)


<h3>1.1.3 Time Series Repeated KFold Cross Validation with vary timesteps - df_no_r3d_r7d</h3>

<b>Dataset: df_no_r3d_r7d:</b> In this dataset the R3D and R7D variables are not considered.

<b>Feature Selection:</b> 

1. The IP, FP and BL variables are discarded.

2. High correlated variables are removed

In [24]:
to_drop = [
    "Data/Hora",
    "IP",
    "FP",
    "BL",
    "CALCITA",
    "DESIDRATAÇÃO",
    "GIPSITA",
    "TIO2",
    "AL2O3",
    "PF",
    "FERRITA",
]

df = df_no_r3d_r7d.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(to_drop, axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [25]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [26]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.040 (0.435)
TIMESTEPS: 3 RMSE: 2.024 (0.452)
TIMESTEPS: 5 RMSE: 1.998 (0.561)
TIMESTEPS: 7 RMSE: 2.038 (0.626)
TIMESTEPS: 10 RMSE: 2.135 (0.388)
TIMESTEPS: 15 RMSE: 2.820 (0.594)
TIMESTEPS: 20 RMSE: 2.673 (0.465)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [27]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [28]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 1.942 (0.222)
TIMESTEPS: 3 RMSE: 2.332 (0.381)
TIMESTEPS: 5 RMSE: 2.680 (0.534)
TIMESTEPS: 7 RMSE: 2.284 (0.473)
TIMESTEPS: 10 RMSE: 2.436 (0.356)
TIMESTEPS: 15 RMSE: 2.017 (0.106)
TIMESTEPS: 20 RMSE: 2.547 (0.275)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [29]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [30]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.718 (0.225)
TIMESTEPS: 3 RMSE: 3.250 (0.314)
TIMESTEPS: 5 RMSE: 3.109 (0.208)
TIMESTEPS: 7 RMSE: 3.406 (0.288)
TIMESTEPS: 10 RMSE: 3.473 (1.395)
TIMESTEPS: 15 RMSE: 3.147 (0.372)
TIMESTEPS: 20 RMSE: 2.901 (0.345)


## 1.2 Dataset: df_r3d_only

<h3>1.2.1 Time Series Repeated KFold Cross Validation - df_r3d_only</h3>

<b>Dataset: df_r3d_only:</b> In this dataset only R3D variable is considered.

<b>Feature Selection:</b> The IP and FP variables are discarded.

In [31]:
df = df_r3d_only.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora', "IP", "FP"], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [32]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [33]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 1.849 (0.308)
TIMESTEPS: 3 RMSE: 2.203 (0.634)
TIMESTEPS: 5 RMSE: 2.276 (0.680)
TIMESTEPS: 7 RMSE: 2.463 (0.477)
TIMESTEPS: 10 RMSE: 2.193 (0.875)
TIMESTEPS: 15 RMSE: 1.850 (0.487)
TIMESTEPS: 20 RMSE: 2.113 (0.787)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [34]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [35]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.645 (0.296)
TIMESTEPS: 3 RMSE: 3.412 (0.272)
TIMESTEPS: 5 RMSE: 4.179 (0.652)
TIMESTEPS: 7 RMSE: 3.122 (0.393)
TIMESTEPS: 10 RMSE: 3.358 (0.500)
TIMESTEPS: 15 RMSE: 2.605 (0.825)
TIMESTEPS: 20 RMSE: 2.064 (0.622)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [36]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [37]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 1.504 (0.232)
TIMESTEPS: 3 RMSE: 2.473 (0.718)
TIMESTEPS: 5 RMSE: 2.383 (0.625)
TIMESTEPS: 7 RMSE: 2.458 (0.612)
TIMESTEPS: 10 RMSE: 1.997 (0.326)
TIMESTEPS: 15 RMSE: 2.595 (0.245)
TIMESTEPS: 20 RMSE: 2.961 (0.787)


<h3>1.2.2 Time Series Repeated KFold Cross Validation with vary timesteps - df_r3d_only</h3>

<b>Dataset: df_r3d_only:</b> In this dataset the R3D and R7D variables are not considered.

<b>Feature Selection:</b> The IP, FP and BL variables are discarded.

In [38]:
df = df_r3d_only.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora', "IP", "FP", "BL"], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [39]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [40]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 1.951 (0.464)
TIMESTEPS: 3 RMSE: 2.200 (0.523)
TIMESTEPS: 5 RMSE: 2.383 (0.778)
TIMESTEPS: 7 RMSE: 2.551 (0.532)
TIMESTEPS: 10 RMSE: 2.125 (0.547)
TIMESTEPS: 15 RMSE: 1.865 (0.366)
TIMESTEPS: 20 RMSE: 2.099 (0.462)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [41]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [42]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.600 (0.288)
TIMESTEPS: 3 RMSE: 3.554 (0.445)
TIMESTEPS: 5 RMSE: 3.709 (0.508)
TIMESTEPS: 7 RMSE: 3.475 (0.263)
TIMESTEPS: 10 RMSE: 3.449 (0.423)
TIMESTEPS: 15 RMSE: 2.798 (0.444)
TIMESTEPS: 20 RMSE: 2.735 (1.859)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [43]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [44]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 1.456 (0.123)
TIMESTEPS: 3 RMSE: 2.493 (0.388)
TIMESTEPS: 5 RMSE: 2.036 (0.326)
TIMESTEPS: 7 RMSE: 2.768 (0.499)
TIMESTEPS: 10 RMSE: 2.369 (0.483)
TIMESTEPS: 15 RMSE: 2.497 (0.434)
TIMESTEPS: 20 RMSE: 3.122 (0.460)


<h3>1.2.3 Time Series Repeated KFold Cross Validation with vary timesteps - df_r3d_only</h3>

<b>Dataset: df_r3d_only:</b> In this dataset the R3D and R7D variables are not considered.

<b>Feature Selection:</b>

1. The IP, FP and BL variables are discarded.

2. High correlated variables are removed

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [45]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [46]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 1.939 (0.396)
TIMESTEPS: 3 RMSE: 2.061 (0.610)
TIMESTEPS: 5 RMSE: 2.489 (1.067)
TIMESTEPS: 7 RMSE: 2.332 (0.401)
TIMESTEPS: 10 RMSE: 2.280 (0.944)
TIMESTEPS: 15 RMSE: 1.761 (0.359)
TIMESTEPS: 20 RMSE: 2.262 (0.649)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [47]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [48]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.594 (0.381)
TIMESTEPS: 3 RMSE: 3.559 (0.360)
TIMESTEPS: 5 RMSE: 3.934 (0.391)
TIMESTEPS: 7 RMSE: 3.433 (0.305)
TIMESTEPS: 10 RMSE: 3.252 (0.543)
TIMESTEPS: 15 RMSE: 2.656 (0.471)
TIMESTEPS: 20 RMSE: 2.015 (0.510)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [49]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [50]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 1.406 (0.134)
TIMESTEPS: 3 RMSE: 2.229 (0.523)
TIMESTEPS: 5 RMSE: 2.515 (0.472)
TIMESTEPS: 7 RMSE: 2.997 (0.400)
TIMESTEPS: 10 RMSE: 2.345 (0.498)
TIMESTEPS: 15 RMSE: 2.605 (0.437)
TIMESTEPS: 20 RMSE: 2.658 (0.490)


## 1.3 Dataset: df_r3d_and_r7d

<h3>1.3.1 Time Series Repeated KFold Cross Validation - df_r3d_and_r7d</h3>

<b>Dataset: df_r3d_and_r7d:</b> In this dataset both R3D and R7D variables are considered.

<b>Feature Selection:</b> The IP and FP variables are discarded.

In [51]:
df = df_r3d_and_r7d.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora', "IP", "FP"], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [52]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [53]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 1.994 (0.370)
TIMESTEPS: 3 RMSE: 2.149 (0.504)
TIMESTEPS: 5 RMSE: 2.481 (0.604)
TIMESTEPS: 7 RMSE: 2.302 (0.551)
TIMESTEPS: 10 RMSE: 2.509 (0.930)
TIMESTEPS: 15 RMSE: 2.016 (0.683)
TIMESTEPS: 20 RMSE: 2.609 (1.478)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [54]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [55]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.617 (0.293)
TIMESTEPS: 3 RMSE: 3.190 (0.409)
TIMESTEPS: 5 RMSE: 3.797 (0.642)
TIMESTEPS: 7 RMSE: 3.626 (0.306)
TIMESTEPS: 10 RMSE: 3.202 (0.316)
TIMESTEPS: 15 RMSE: 2.531 (0.656)
TIMESTEPS: 20 RMSE: 1.660 (0.315)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [56]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [57]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 1.642 (0.171)
TIMESTEPS: 3 RMSE: 2.026 (0.408)
TIMESTEPS: 5 RMSE: 2.632 (0.476)
TIMESTEPS: 7 RMSE: 2.661 (0.649)
TIMESTEPS: 10 RMSE: 2.460 (0.662)
TIMESTEPS: 15 RMSE: 2.604 (0.588)
TIMESTEPS: 20 RMSE: 2.805 (0.634)


<h3>1.3.2 Time Series Repeated KFold Cross Validation - df_r3d_and_r7d</h3>

<b>Dataset: df_r3d_and_r7d:</b> In this dataset both R3D and R7D variables are considered.

<b>Feature Selection:</b> The IP, FP, and BL variables are discarded.

In [58]:
df = df_r3d_and_r7d.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora', "IP", "FP", "BL"], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [59]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [60]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 1.836 (0.361)
TIMESTEPS: 3 RMSE: 2.195 (0.589)
TIMESTEPS: 5 RMSE: 2.464 (0.704)
TIMESTEPS: 7 RMSE: 2.193 (0.529)
TIMESTEPS: 10 RMSE: 2.427 (0.947)
TIMESTEPS: 15 RMSE: 2.187 (0.528)
TIMESTEPS: 20 RMSE: 2.095 (0.409)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [61]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [62]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 2.519 (0.152)
TIMESTEPS: 3 RMSE: 3.342 (0.463)
TIMESTEPS: 5 RMSE: 3.823 (0.328)
TIMESTEPS: 7 RMSE: 3.485 (0.192)
TIMESTEPS: 10 RMSE: 3.155 (0.377)
TIMESTEPS: 15 RMSE: 2.357 (0.715)
TIMESTEPS: 20 RMSE: 2.029 (0.528)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [63]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [64]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 1.645 (0.167)
TIMESTEPS: 3 RMSE: 2.441 (0.627)
TIMESTEPS: 5 RMSE: 2.562 (0.663)
TIMESTEPS: 7 RMSE: 2.801 (0.502)
TIMESTEPS: 10 RMSE: 2.600 (0.747)
TIMESTEPS: 15 RMSE: 2.737 (0.760)
TIMESTEPS: 20 RMSE: 3.277 (0.601)


<h3>1.3.3 Time Series Repeated KFold Cross Validation - df_r3d_and_r7d</h3>

<b>Dataset: df_r3d_and_r7d:</b> In this dataset both R3D and R7D variables are considered.

<b>Feature Selection:</b>
    1. The IP, FP and BL variables are discarded.
    2. High correlated variables are removed

In [65]:
to_drop = [
    "Data/Hora",
    "IP",
    "FP",
    "BL",
    "CALCITA",
    "DESIDRATAÇÃO",
    "GIPSITA",
    "TIO2",
    "AL2O3",
    "PF",
    "FERRITA",
]

df = df_r3d_and_r7d.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(to_drop, axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [66]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [67]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 1.541 (0.346)
TIMESTEPS: 3 RMSE: 2.051 (0.492)
TIMESTEPS: 5 RMSE: 2.156 (0.421)
TIMESTEPS: 7 RMSE: 2.239 (0.847)
TIMESTEPS: 10 RMSE: 2.258 (0.602)
TIMESTEPS: 15 RMSE: 1.816 (0.665)
TIMESTEPS: 20 RMSE: 1.906 (0.317)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [68]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [69]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 1.865 (0.209)
TIMESTEPS: 3 RMSE: 3.181 (0.388)
TIMESTEPS: 5 RMSE: 3.418 (0.402)
TIMESTEPS: 7 RMSE: 3.566 (0.398)
TIMESTEPS: 10 RMSE: 2.546 (0.265)
TIMESTEPS: 15 RMSE: 1.900 (0.484)
TIMESTEPS: 20 RMSE: 2.149 (1.088)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [70]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [71]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, repeats, timesteps_list, True
)

TIMESTEPS: 1 RMSE: 1.502 (0.194)
TIMESTEPS: 3 RMSE: 2.135 (0.442)
TIMESTEPS: 5 RMSE: 2.292 (0.421)
TIMESTEPS: 7 RMSE: 2.127 (0.387)
TIMESTEPS: 10 RMSE: 2.616 (0.488)
TIMESTEPS: 15 RMSE: 2.702 (0.724)
TIMESTEPS: 20 RMSE: 3.127 (0.457)
